In [11]:
import time

from squid_py.did import id_to_did, did_to_id
from squid_py.keeper.web3_provider import Web3Provider
from ocean_cli.api.conditions import access, release_reward, check_permissions
from ocean_cli.ocean import get_ocean

ocean = get_ocean('bob.ini')

template = ocean.keeper.escrow_access_secretstore_template.get_instance()
lockreward = ocean.keeper.lock_reward_condition.get_instance()

filters = [
    template.events.AgreementCreated.createFilter(fromBlock='latest'),
    lockreward.events.Fulfilled.createFilter(fromBlock='latest'),
]


def handle_access(event):
    agreement_id = event['args'].get('_agreementId', None)
    access_consumer = event['args'].get('_accessConsumer', None)
    access_provider = event['args'].get('_accessProvider', None)
    
    if agreement_id and access_consumer and access_provider == ocean.account.address:   
        agreement_id = Web3Provider.get_web3().toHex(agreement_id)
        did_0x = ocean.agreements.get(agreement_id).did
        did = id_to_did(did_0x)
        print('Agreement', agreement_id)
        try:
#             response = access(ocean, ocean.account, agreement_id, access_consumer)
            response = ocean.keeper.access_secret_store_condition.fulfill(
                agreement_id, 
                did_0x, 
                access_consumer, 
                ocean.account
            )

            print(response, check_permissions(ocean, None, did, access_consumer))
            return response
        except ValueError as e:
            print('ValueError', e)
        return False
    


while True:
    for _filter in filters:
        for event in _filter.get_new_entries():
            print(event['event'], event)
            if event['event'] == 'AgreementCreated':
                time.sleep(3)
                handle_access(event)
            elif event['event'] == 'Fulfilled':
                release_reward(ocean, ocean.account, event['args'].get('_agreementId', None))
    time.sleep(0.1)

2019-06-25 17:48:45 dimi-xps squid_py.keeper.diagnostics[9980] INFO Keeper contract artifacts (JSON abi files) at: /home/dimi/oceanprotocol/ocean-cli-py/artifacts
2019-06-25 17:48:45 dimi-xps squid_py.keeper.diagnostics[9980] INFO Using keeper contracts from network spree, network id is 8996
2019-06-25 17:48:45 dimi-xps squid_py.keeper.diagnostics[9980] INFO Looking for keeper contracts ending with ".spree.json", e.g. "AgreementStoreManager.spree.json".
2019-06-25 17:48:45 dimi-xps root[9980] INFO Finished loading keeper contracts:
	Dispenser: 0xF152cF3c67dFD41a317eAe8fAc0e1e8E98724A13
	OceanToken: 0x726baA2f854A3BEC2378a707AeB38c9d933Ebad6
	DIDRegistry: 0xc354ba9AD5dF1023C2640b14A09E61a500F21546
	AgreementStoreManager: 0x645439117eB378a6d35148452E287a038666Ed67
	TemplateStoreManager: 0xfeA10BBb093d7fcb1EDf575Aa7e28d37b9DcFcE9
	ConditionStoreManager: 0x336EFb3c9E56F713dFdA4CDB3Dd0882F3226b6eE
	AccessSecretStoreCondition: 0x19513460bc16254c74AE806683E906478A42B543
	SignCondition: 0xEEE5

AgreementCreated AttributeDict({'args': AttributeDict({'_agreementId': b'\x01\x8d\xd9?5\nK7\x87t\xe4\x0c"1V\xc8\xfe5\x8b\xd02\x01F \xa4\xa3v\nl\xf8U\xf4', '_accessConsumer': '0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0', '_accessProvider': '0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e', '_did': b'\x1c$\x10\x15K\xf2I\xa7\x9d\xb0v7" \x04v\x84(\xa1\xf3\xb7fD\xd3\x82\xb3\xd0\xc9\xc2\x1d;\t', '_timeLocks': [0, 0, 0], '_timeOuts': [0, 0, 0]}), 'event': 'AgreementCreated', 'logIndex': 4, 'transactionIndex': 1, 'transactionHash': HexBytes('0x2d2d038d01f0348d53bca6608887119a83bff2d8eabd90993fb92eccf4a895af'), 'address': '0xD306b5edCDC7819E1EB80B43De6548931706A3f4', 'blockHash': HexBytes('0x016015b5f1185ddae07456b7c419863f0afcad5d653be4f2659c939a4a2c9c4f'), 'blockNumber': 763})
Agreement 0x018dd93f350a4b378774e40c223156c8fe358bd032014620a4a3760a6cf855f4
True True


KeyboardInterrupt: 

In [ ]:
ocean.baln